In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from rl.agents import DQNAgent
import gym
from gym import spaces
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

In [ ]:

nc = data.columns.difference(['state', 'MDC', 'Fachabteilung', 'Sekundär','Fachabteilung_encoded','Sekundär_encoded','state_encoded','MDC_encoded'])
nc.shape

(285,)

In [2]:
years_files = {
    2021: "Data/merged_2021.xlsx",
    2022: "Data/merged_2022.xlsx",
    2023: "Data/merged_2023.xlsx"
}
dfs = []

for year, file_path in years_files.items():
    df = pd.read_excel(file_path, engine='openpyxl')
    df['year'] = year
    dfs.append(df)
    
data = pd.concat(dfs, ignore_index=True)
label_encoders = {}
categorical_cols = ['state', 'MDC', 'Fachabteilung', 'Sekundär']
for col in categorical_cols:
    le = LabelEncoder()
    data[col + '_encoded'] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le  # Save encoders for later use

# Normalize numerical columns (e.g., 'total_patients')
scaler = StandardScaler()
numerical_cols = data.columns.difference(['state', 'MDC', 'Fachabteilung', 'Sekundär','Fachabteilung_encoded','Sekundär_encoded','state_encoded','MDC_encoded'])
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])
# numerical_cols.append(["State","MDC"])


C:\Users\Ali\AppData\Local\Temp\ipykernel_13832\1759614095.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[col + '_encoded'] = le.fit_transform(data[col].astype(str))
C:\Users\Ali\AppData\Local\Temp\ipykernel_13832\1759614095.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[col + '_encoded'] = le.fit_transform(data[col].astype(str))
C:\Users\Ali\AppData\Local\Temp\ipykernel_13832\1759614095.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [ ]:
class ReshapeProcessor(Processor):
    def process_state_batch(self, state_batch):
        # Remove the extra dimension: (batch, 1, 2) → (batch, 2)
        return np.squeeze(state_batch, axis=1)
    
class HospitalEnv(gym.Env):
    def __init__(self, data):
        super(HospitalEnv, self).__init__()
        self.data = data
        self.state_size = 285  # patients, state
        self.action_space = spaces.Discrete(27)  # 27 MDCs
        self.observation_space = spaces.Box(
            low=np.array([0, 0]), 
            high=np.array([1, 1]),
            dtype=np.float32
        )
        
    def _get_state(self):
        # Extract ONLY the two features we need
        out = self.current_patient[numerical_cols].values.flatten()
        return out
    
    def reset(self):
        # Randomly select a patient case
        self.current_patient = self.data.sample(1)
        return self._get_state()
    
    def step(self, action):
        # Simulate reward (e.g., correct department = +1, wrong = -1)
        correct_mdc = self.current_patient['MDC_encoded'].values[0]
        reward = 1 if action == correct_mdc else -1
        done = True  # One-step episode
        return self.reset(), reward, done, {}

In [24]:
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=50000, window_length=1)
model = Sequential([
    Dense(24, activation='relu', input_shape=(2,)),
    Dense(24, activation='relu'),
    Dense(27, activation='linear')  # 27 actions (MDCs)
])
# Add the processor to your agent
agent = DQNAgent(
    model=model,
    policy=policy,
    memory=memory,
    nb_actions=27,
    processor=ReshapeProcessor()  # Fixes shape issues
)

agent.compile(optimizer='adam')

# Train the agent
env = HospitalEnv(data)
agent.fit(env, nb_steps=10000, visualize=False, verbose=1)

c:\Users\Ali\miniconda3\envs\conv\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
c:\Users\Ali\miniconda3\envs\conv\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Model input shape: (None, 2)
Training for 10000 steps ...
Interval 1 (0 steps performed)
  993/10000 [=>............................] - ETA: 38s - reward: -0.9255

c:\Users\Ali\miniconda3\envs\conv\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 164s 16ms/step - reward: -0.8724
done, took 163.779 seconds


In [25]:
# Save the trained model
agent.save_weights('dqn_hospital_weights.h5f', overwrite=True)

# Test the agent
test_obs = env.reset()
action = agent.forward(test_obs)
print(f"Recommended Department: {label_encoders['MDC'].inverse_transform([action])}")

Recommended Department: ['04 – Krankheiten und Störungen der Atmungsorgane']


In [54]:
env = HospitalEnv(data)
state = env.reset()
print("State shape:", state.shape)  # Should be (2,)
print("State values:", state)

State shape: (2,)
State values: [-0.27857806 14.        ]


c:\Users\Ali\miniconda3\envs\conv\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
